In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



In [2]:
referendum = pd.read_csv("./data/referendum.csv", ";")
regions = pd.read_csv("./data/regions.csv", ",")
departments = pd.read_csv("./data/departments.csv", ",")

In [16]:
referendum['Department code'] = referendum['Department code'].apply(lambda x: x.zfill(2))
referendum.head()

,Department code,Department name,Town code,Town name,Registered,Abstentions,Null,Choice A,Choice B
0,01,AIN,1,L'Abergement-Clémenciat,592,84,9,154,345
1,01,AIN,2,L'Abergement-de-Varey,215,36,5,66,108
2,01,AIN,4,Ambérieu-en-Bugey,8205,1698,126,2717,3664
3,01,AIN,5,Ambérieux-en-Dombes,1152,170,18,280,684
4,01,AIN,6,Ambléon,105,17,1,35,52


In [5]:
regions_and_departments = pd.merge(regions, departments, left_on='code', right_on='region_code', how='inner'
             ).drop(
                ['code_x','id_x', 'slug_x', 'slug_y', 'id_y'], axis=1
            ).rename(columns=
                {'region_code':'code_reg', 'name_x':'name_reg','code_y':'code_dep','name_y':'name_dep'}
            )

regions_and_departments.sort_values(by='code_dep', ascending= True, inplace=True)



regions_and_departments.head()

,name_reg,code_reg,code_dep,name_dep
81,Auvergne-Rhône-Alpes,84,01,Ain
32,Hauts-de-France,32,02,Aisne
82,Auvergne-Rhône-Alpes,84,03,Allier
93,Provence-Alpes-Côte d'Azur,93,04,Alpes-de-Haute-Provence
94,Provence-Alpes-Côte d'Azur,93,05,Hautes-Alpes


In [24]:
merge_referendum_and_areas = pd.merge(regions_and_departments, referendum, left_on='code_dep', right_on='Department code', how='inner')


print(len(merge_referendum_and_areas))

merge_referendum_and_areas.dropna(inplace=True)
print(len(merge_referendum_and_areas))

    referendum_and_areas.groupby(['code_reg', 'name_reg'], as_index=False)[
        ['Registered', 'Abstentions', 'Null', 'Choice A', 'Choice B']
        ].sum()
    referendum_and_areas.set_index('code_reg')


merge_referendum_and_areas

36565
36565


,name_reg,Registered,Abstentions,Null,Choice A,Choice B
code_reg,,,,,,
11,Île-de-France,6942361,1515884,85997,2536620,2803860
24,Centre-Val de Loire,1801366,326560,28827,590890,855089
27,Bourgogne-Franche-Comté,1997925,358321,33671,668246,937687
28,Normandie,2372779,426075,35513,816972,1094219
32,Hauts-de-France,4210444,866555,60601,1411257,1872031
44,Grand Est,3857089,783348,56951,1088684,1928106
52,Pays de la Loire,2605438,412509,47677,922468,1222784
53,Bretagne,2380594,364724,36077,944052,1035741
75,Nouvelle-Aquitaine,4201774,711017,71629,1641249,1777879


In [13]:

import geopandas as gpd
import matplotlib.pyplot as plt


In [8]:
geo_reg = gpd.read_file('./data/regions.geojson')
geo_reg.head()




,code,nom,geometry
0,11,Île-de-France,"POLYGON ((1.92215 48.45760, 1.92074 48.44775, ..."
1,24,Centre-Val de Loire,"POLYGON ((0.81482 48.67016, 0.82767 48.68072, ..."
2,27,Bourgogne-Franche-Comté,"POLYGON ((6.94054 47.43337, 6.94168 47.41580, ..."
3,28,Normandie,"POLYGON ((-1.11962 49.35557, -1.11503 49.36240..."
4,32,Hauts-de-France,"POLYGON ((4.14089 49.97876, 4.15398 49.97484, ..."


In [9]:
geo_referundum = pd.merge(geo_reg, merge_referendum_and_areas, left_on='code', right_on='code_reg', how='inner')
geo_referundum['choice_A_ratio'] = geo_referundum['Choice A']/geo_referundum['Choice B']